In [363]:
import pandas as pd
import numpy as np
import math
import json

In [364]:
from chardet.universaldetector import UniversalDetector
detector = UniversalDetector()
with open('cities.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
print(detector.result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [365]:
cities = pd.read_csv('cities.csv', encoding = 'utf-8')
cities.sample(5)

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
622,606803.0,обл,Нижегородская,р-н,Уренский,г,Урень,NaN,NaN,5204500100000,...,4: город,1,22254501000,2.265410e+10,5235,UTC+3,57.461255,45.783394,Приволжский,12311
749,347210.0,обл,Ростовская,р-н,Морозовский,г,Морозовск,NaN,NaN,6102500100000,...,4: город,1,60234501000,6.063410e+10,6192,UTC+3,48.351172,41.830801,Южный,27644
959,636615.0,обл,Томская,NaN,NaN,г,Кедровый,NaN,NaN,7000000200000,...,4: город,0,69407000000,6.970700e+10,7000,UTC+7,57.561846,79.567891,Сибирский,2451
178,368000.0,Респ,Дагестан,NaN,NaN,г,Хасавюрт,NaN,NaN,500000900000,...,4: город,0,82435000000,8.273500e+10,544,UTC+3,43.250519,46.585228,Северо-Кавказский,133929
718,692851.0,край,Приморский,NaN,NaN,г,Партизанск,NaN,NaN,2500000500000,...,4: город,0,5417000000,5.717000e+06,2509,UTC+10,43.127983,133.126475,Дальневосточный,38648


In [366]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Индекс                             1108 non-null   float64
 1   Тип региона                        1112 non-null   object 
 2   Регион                             1112 non-null   object 
 3   Тип района                         629 non-null    object 
 4   Район                              629 non-null    object 
 5   Тип города                         1108 non-null   object 
 6   Город                              1108 non-null   object 
 7   Тип н/п                            2 non-null      object 
 8   Н/п                                2 non-null      object 
 9   Код КЛАДР                          1112 non-null   int64  
 10  Код ФИАС                           1112 non-null   object 
 11  Уровень по ФИАС                    1112 non-null   objec

In [367]:
cities.columns

Index(['Индекс', 'Тип региона', 'Регион', 'Тип района', 'Район', 'Тип города',
       'Город', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Часовой пояс', 'Широта', 'Долгота', 'Федеральный округ',
       'Население'],
      dtype='object')

In [368]:
cities.drop(['Индекс', 'Тип региона', 'Тип н/п', 'Н/п', 'Код КЛАДР', 'Код ФИАС', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Код ОКАТО', 'Код ОКТМО',
       'Код ИФНС', 'Федеральный округ'], axis = 1, inplace = True)

In [373]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1108 entries, 0 to 1111
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Регион        1108 non-null   object 
 1   Тип района    628 non-null    object 
 2   Район         628 non-null    object 
 3   Тип города    1108 non-null   object 
 4   Город         1108 non-null   object 
 5   Часовой пояс  1108 non-null   object 
 6   Широта        1108 non-null   float64
 7   Долгота       1108 non-null   float64
 8   Население     1108 non-null   object 
dtypes: float64(2), object(7)
memory usage: 86.6+ KB


In [371]:
cities.dropna(subset = ['Город'], inplace = True)

In [372]:
cities[cities['Город'] == 'Казань']

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население
924,Татарстан,NaN,NaN,г,Казань,UTC+3,55.794388,49.111531,1216965


In [374]:
cities['Часовой пояс'] = cities['Часовой пояс'].str.replace('UTC+', '')#удаляю UTC из часового пояса

In [375]:
cities['Часовой пояс'] = cities['Часовой пояс'].str.replace(' ', '')

In [376]:
cities['Часовой пояс'] = cities['Часовой пояс'].apply(lambda x: np.int(x))

In [377]:
cities['Часовой пояс'].unique()

array([ 3,  7,  9,  4,  5,  8, 10,  2, 12, 11,  6], dtype=int64)

In [378]:
cities[cities['Город'] == 'Белогорск']['Часовой пояс']

15     9
423    3
Name: Часовой пояс, dtype: int64

In [412]:
cities['Город+Регион'] = cities['Город'] + ' / ' + cities['Регион']
cities['Город+Регион'].value_counts()

Малоархангельск / Орловская    1
Гвардейск / Калининградская    1
Красный Холм / Тверская        1
Данков / Липецкая              1
Черепаново / Новосибирская     1
                              ..
Верхоянск / Саха /Якутия/      1
Курлово / Владимирская         1
Оханск / Пермский              1
Петухово / Курганская          1
Апшеронск / Краснодарский      1
Name: Город+Регион, Length: 1107, dtype: int64

In [380]:
cities[cities['Город+Регион'] =='ЯлтаКрым']

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население,Город+Регион
419,Крым,NaN,NaN,г,Ялта,3,44.416366,34.044550,8497,ЯлтаКрым
434,Крым,NaN,NaN,г,Ялта,3,44.495195,34.166386,78040,ЯлтаКрым


In [382]:
cities.drop(index=[419], inplace = True)

In [404]:
cities['Город+Регион'].value_counts()

СарапулУдмуртская            1
ТюменьТюменская              1
АрхангельскАрхангельская     1
МикуньКоми                   1
ЗарайскМосковская            1
                            ..
ПравдинскКалининградская     1
Катав-ИвановскЧелябинская    1
КлинцыБрянская               1
УрюпинскВолгоградская        1
КраснослободскМордовия       1
Name: Город+Регион, Length: 1107, dtype: int64

In [405]:
for i in cities['Город+Регион']:
    #print(i)
    int(cities[cities['Город+Регион'] == i]['Часовой пояс'].values)+2.5

In [394]:
#Отсекаем часовые пояса +- 2 часа
near_Kazan = cities[cities['Часовой пояс'].between(float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)-2.5,
                                            float(cities[cities['Город'] == 'Самара']['Часовой пояс'].values)+2.5,
                                            inclusive = True)]

In [395]:
# Отсекаем города по Долготе
near_Kazan = near_Kazan[near_Kazan['Долгота'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)-1,
                                            float(near_Kazan[near_Kazan['Город'] == 'Самара']['Долгота'].values)+1,
                                            inclusive = True)]

In [396]:
math.cos(round(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values), 6) * 0.017)

0.618221918105964

In [397]:
max_latitude = 40000/360 * math.cos(round(float(near_Kazan[near_Kazan['Город'] == 'Самара']['Широта'].values), 6) * 0.017)
print(max_latitude)

68.69132423399601


In [398]:
latitude = round(111.111/(max_latitude/60)/60, 6)
latitude

1.617541

In [399]:
near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)-latitude,
                                                     float(near_Kazan[near_Kazan['Город'] == 'Самара']
                                                           ['Широта'].values)+latitude,
                                            inclusive = True)]

In [400]:
near_Kazan

,Регион,Тип района,Район,Тип города,Город,Часовой пояс,Широта,Долгота,Население,Город+Регион
771,Самарская,NaN,NaN,г,Жигулевск,4,53.401155,49.494529,57565,ЖигулевскСамарская
772,Самарская,NaN,NaN,г,Кинель,4,53.220999,50.634304,34472,КинельСамарская
774,Самарская,NaN,NaN,г,Новокуйбышевск,4,53.099530,49.947766,108449,НовокуйбышевскСамарская
778,Самарская,NaN,NaN,г,Самара,4,53.195166,50.106769,1164900,СамараСамарская
780,Самарская,NaN,NaN,г,Тольятти,4,53.520644,49.389461,719484,ТольяттиСамарская
781,Самарская,NaN,NaN,г,Чапаевск,4,52.928966,49.867422,72689,ЧапаевскСамарская
932,Татарстан,р-н,Нурлатский,г,Нурлат,3,54.428158,50.805043,32600,НурлатТатарстан
1000,Ульяновская,NaN,NaN,г,Димитровград,4,54.216845,49.626165,122549,ДимитровградУльяновская


In [401]:
near_sity_s = dict()

near_sity_s['Казань'] = list()

for i in near_Kazan['Город']:
    near_sity_s['Казань'].append(i)

print(near_sity_s) 

{'Казань': ['Жигулевск', 'Кинель', 'Новокуйбышевск', 'Самара', 'Тольятти', 'Чапаевск', 'Нурлат', 'Димитровград']}


In [219]:
sev_city = ['Казань', 'Самара', 'Ульяновск', 'Альметьевск']

In [233]:
several_city = cities.query('Город in ["Казань","Самара"]')

In [406]:
def near_city(df, distance):
    near_sity_s = dict() #создаю словарь, что бы добавить города
    
    for i in df['Город+Регион']:
        near_sity_s[i] = list()
    
        #Отсекаем часовые пояса +- 2 часа
        near_Kazan = df[df['Часовой пояс'].between(float(df[df['Город+Регион'] == i]['Часовой пояс'].values)-2.5,
                                                float(df[df['Город+Регион'] == i ]['Часовой пояс'].values)+2.5,
                                                inclusive = True)]
    
        # Отсекаем города по Долготе
        longitude = distance/111.111
        near_Kazan = near_Kazan[near_Kazan['Долгота'].between(float(near_Kazan[near_Kazan['Город+Регион'] == i]
                                                            ['Долгота'].values)-longitude,
                                                float(near_Kazan[near_Kazan['Город+Регион'] == i]
                                                      ['Долгота'].values)+longitude,
                                                inclusive = True)]
    
        latitude = 40000/360 * math.cos(float(near_Kazan[near_Kazan['Город+Регион'] == i]['Широта'].values)*0.017)
        latitude = round(distance/(latitude/60)/60, 6)
    
        # Отсекаем города по Широте
        near_Kazan = near_Kazan[near_Kazan['Широта'].between(float(near_Kazan[near_Kazan['Город+Регион'] == i]
                                                               ['Широта'].values)-latitude,
                                                         float(near_Kazan[near_Kazan['Город+Регион'] == i]
                                                               ['Широта'].values)+latitude,
                                                inclusive = True)]
    
        for j in near_Kazan['Город+Регион']:
            near_sity_s[i].append(j)

    return near_sity_s

Отсекаем часовые пояса +\- 2 часа
Считам длину долготы - разница +\- 111 км
Длина 1 градуса широты 40000\*cos(широты) - 1 градус - длина широты \ 360, в градусе 60 минут - 
   пересчитываем в десятичные доли - 1 градус + 1 мин\60
Находим по разницы широты +\- 1 градус, долготы расчётная разница, список городов
Выводим списко городов

In [413]:
summary = near_city(cities, 50)

In [251]:
print(summary)

{'Самара': ['Жигулевск', 'Кинель', 'Новокуйбышевск', 'Самара', 'Тольятти', 'Чапаевск', 'Нурлат', 'Димитровград'], 'Казань': ['Уржум', 'Волжск', 'Арск', 'Болгар', 'Буинск', 'Зеленодольск', 'Иннополис', 'Казань', 'Лаишево', 'Тетюши', 'Ульяновск', 'Козловка']}


In [414]:
jsonData = json.dumps(summary)

In [415]:
with open("json_near_city.json", "w") as file:
    file.write(jsonData)

In [416]:
with open('json_near_city.json') as f:
   expl = json.load(f)

In [417]:
from pprint import pprint  
pprint(expl)

{'Абаза / Хакасия': ['Абаза / Хакасия'],
 'Абакан / Хакасия': ['Минусинск / Красноярский',
                      'Абакан / Хакасия',
                      'Саяногорск / Хакасия',
                      'Черногорск / Хакасия'],
 'Абдулино / Оренбургская': ['Абдулино / Оренбургская', 'Бавлы / Татарстан'],
 'Абинск / Краснодарский': ['Абинск / Краснодарский',
                            'Геленджик / Краснодарский',
                            'Крымск / Краснодарский',
                            'Новороссийск / Краснодарский',
                            'Славянск-на-Кубани / Краснодарский'],
 'Агидель / Башкортостан': ['Агидель / Башкортостан',
                            'Нефтекамск / Башкортостан',
                            'Камбарка / Удмуртская',
                            'Сарапул / Удмуртская'],
 'Агрыз / Татарстан': ['Агрыз / Татарстан', 'Ижевск / Удмуртская'],
 'Адыгейск / Адыгея': ['Адыгейск / Адыгея',
                       'Горячий Ключ / Краснодарский',
                    

 'Дорогобуж / Смоленская': ['Дорогобуж / Смоленская',
                            'Ельня / Смоленская',
                            'Сафоново / Смоленская'],
 'Дрезна / Московская': ['Александров / Владимирская',
                         'Карабаново / Владимирская',
                         'Киржач / Владимирская',
                         'Покров / Владимирская',
                         'Струнино / Владимирская',
                         'Воскресенск / Московская',
                         'Дрезна / Московская',
                         'Егорьевск / Московская',
                         'Коломна / Московская',
                         'Куровское / Московская',
                         'Ликино-Дулево / Московская',
                         'Ногинск / Московская',
                         'Орехово-Зуево / Московская',
                         'Павловский Посад / Московская',
                         'Электрогорск / Московская',
                         'Электросталь / Московская'],
 'Д

                        'Фурманов / Ивановская',
                        'Шуя / Ивановская',
                        'Волгореченск / Костромская'],
 'Красавино / Вологодская': ['Котлас / Архангельская',
                             'Сольвычегодск / Архангельская',
                             'Великий Устюг / Вологодская',
                             'Красавино / Вологодская'],
 'Красноармейск / Московская': ['Красноармейск / Московская',
                                'Красноармейск / Саратовская',
                                'Саратов / Саратовская',
                                'Энгельс / Саратовская'],
 'Красноармейск / Саратовская': ['Красноармейск / Московская',
                                 'Красноармейск / Саратовская',
                                 'Саратов / Саратовская',
                                 'Энгельс / Саратовская'],
 'Красновишерск / Пермский': ['Красновишерск / Пермский',
                              'Соликамск / Пермский'],
 'Красногорск / Моско

                                   'Михайловск / Ставропольский',
                                   'Невинномысск / Ставропольский',
                                   'Ставрополь / Ставропольский'],
 'Невьянск / Свердловская': ['Верхний Тагил / Свердловская',
                             'Верхняя Пышма / Свердловская',
                             'Верхняя Салда / Свердловская',
                             'Дегтярск / Свердловская',
                             'Екатеринбург / Свердловская',
                             'Кировград / Свердловская',
                             'Невьянск / Свердловская',
                             'Нижний Тагил / Свердловская',
                             'Новоуральск / Свердловская',
                             'Первоуральск / Свердловская',
                             'Ревда / Свердловская',
                             'Среднеуральск / Свердловская'],
 'Нелидово / Тверская': ['Белый / Тверская', 'Нелидово / Тверская'],
 'Неман / Калининградска

                        'Черемхово / Иркутская'],
 'Свободный / Амурская': ['Белогорск / Амурская',
                          'Свободный / Амурская',
                          'Циолковский / Амурская',
                          'Шимановск / Амурская'],
 'Себеж / Псковская': ['Опочка / Псковская', 'Себеж / Псковская'],
 'Северо-Курильск / Сахалинская': ['Северо-Курильск / Сахалинская'],
 'Северобайкальск / Бурятия': ['Северобайкальск / Бурятия'],
 'Северодвинск / Архангельская': ['Северодвинск / Архангельская'],
 'Североморск / Мурманская': ['Гаджиево / Мурманская',
                              'Кола / Мурманская',
                              'Мурманск / Мурманская',
                              'Оленегорск / Мурманская',
                              'Полярный / Мурманская',
                              'Североморск / Мурманская',
                              'Снежногорск / Мурманская'],
 'Североуральск / Свердловская': ['Волчанск / Свердловская',
                                

                               'Егорьевск / Московская',
                               'Куровское / Московская',
                               'Ликино-Дулево / Московская',
                               'Ногинск / Московская',
                               'Орехово-Зуево / Московская',
                               'Павловский Посад / Московская',
                               'Черноголовка / Московская',
                               'Электрогорск / Московская',
                               'Электросталь / Московская'],
 'Электросталь / Московская': ['Александров / Владимирская',
                               'Карабаново / Владимирская',
                               'Киржач / Владимирская',
                               'Струнино / Владимирская',
                               'Бронницы / Московская',
                               'Воскресенск / Московская',
                               'Дрезна / Московская',
                               'Жуковский / Московская',
   